In [1]:
#Creates a Jupyter image that will be our display surface
#format can be 'jpeg' or 'png';specify width and height to set a viewer size
import ipywidgets as widgets 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
jup_img=widgets.Image(format='jpeg',width=500,height=500)

In [2]:
# Boilerplate to create a Ginga viewer connected to this widget
# this could be simplified, or hidden behind a class or convenience 
# method
# NOTE: you don't have to specify a log file--and if you are not interested
# in the log just specify null=True for a null logger
# level=10 will give you the most debugging information
from ginga.misc.log import get_logger
logger=get_logger("my viewer",log_stderr=False,log_file='/tmp/ginga.log',level=40)

from ginga.web.jupyterw.ImageViewJpw import EnhancedCanvasView
v1=EnhancedCanvasView(logger=logger)
#links the jupyter widget and ginga
v1.set_widget(jup_img)
#enables all possible keyboard and pointer operations
bd=v1.get_bindings()
bd.enable_all(True)

In [3]:
coordinates=widgets.HTML()
#callback to display position in RA/DEC deg
def mouse_move(viewer,button,data_x,data_y,w):
    image=viewer.get_image()
    if image is not None:
        ra,dec=image.pixtoradec(data_x,data_y)
        w.value="cursor at %f,%f" % (ra,dec)
v1.add_callback('cursor-changed',mouse_move,coordinates)

In [4]:
%%javascript
IPython.OutputArea.prototype._should_scroll=function(lines){return false;}

<IPython.core.display.Javascript object>

In [5]:
#viewer embedded here
widgets.VBox([jup_img,coordinates])

In [7]:
#Loads a FITS file. The above viewer should update.
v1.load_fits("FITS file goes here")
v1.scale_to(.8,.8)

 [astropy.io.fits.verify]


In [ ]:
#Draw circles around all objects
canvas=v1.add_canvas()
Circle=canvas.get_draw_class('circle')
for i in range(30):
    x,y=X[i],Y[i]
    canvas.add(Circle(x,y,radius=15,color='yellow'))

In [6]:
#this code loads the DCMP Table generated from the ziggy.ucolick.org pipeline.
from astropy.io import ascii,fits

filename = 'dcmp table here'

dcmp_table = ascii.read(filename, data_start = 1)
header = fits.open(filename)[0].header

for key in dcmp_table.keys():
    colname = header[key.replace('col','COLTBL')]
    dcmp_table.rename_column(key, colname)
    
#We want to grab the X and Y positions of the potential transients detected
X=np.array(dcmp_table['Xpos'])
Y=np.array(dcmp_table['Ypos'])

In [10]:
#Saves an image for each potential optical transient to a filepath
#centered based on X,Y coordinates provided by DCMP Table
import logging
logging.basicConfig(format='%(levelname)s: %(name)s(%(funcName)s): %(message)s', level=logging.DEBUG)
for i in range(30):
    v1.set_pan(X[i],Y[i])
    v1.save_rgb_image_as_file("your_filepath/image_{}_{}.jpg".format(X[i],Y[i]),format='jpeg',quality=90)
